## 作業

閱讀以下兩篇文獻，了解隨機森林原理，並試著回答後續的思考問題
- [隨機森林 (random forest) - 中文](http://hhtucode.blogspot.tw/2013/06/ml-random-forest.html)
- [how random forest works - 英文](https://medium.com/@Synced/how-random-forest-algorithm-works-in-machine-learning-3c0fe15b6674)


1. 隨機森林中的每一棵樹，是希望能夠

    - 沒有任何限制，讓樹可以持續生長 (讓樹生成很深，讓模型變得複雜)
    
    - 不要過度生長，避免 Overfitting
    
    
2. 假設總共有 N 筆資料，每棵樹用取後放回的方式抽了總共 N 筆資料生成，請問這棵樹大約使用了多少 % 不重複的原資料生成?
hint: 0.632 bootstrap


有n个样本，我们有放回的随机从中抽取n次。

在第一次抽取时，样本A被选中的概率是 $\frac{1}{n}$

，不被选中的概率自然就是$(1-\frac{1}{n})$。每次抽取都是独立的，所以当抽完n次之后，A一次都没有被抽中的概率就是

$$(1-\frac{1}{n})$^n$.

这个式子眼熟吗？这个就是高等数学中那个著名的极限

$\lim_{n→∞}(1-\frac{1}{n})= \frac{1}{e}$.

所以当bootstrap样本总数很大的时候，任意一个样本被抽中的概率就是

$1−\frac{1}{e}≈1−\frac{1}{2.71828}≈0.632$。


I will get to the 0.632 estimator, but it'll be a somewhat long development:

Suppose we want to predict Y with X using the function f, where f may depend on some parameters that are estimated using the data (Y,X), e.g. f(X)=Xβ

A naïve estimate of prediction error is


$ err^n-=1N∑i=1NL(yi,f(xi))$
$ MSE = \frac{1}{n}\sum_{i=1}^{n}{(Y_i - \hat{Y}_i)^2} $

rr¯¯¯¯¯¯¯=1N∑i=1NL(yi,f(xi)
where L is some loss function, e.g. squared error loss. This is often called training error. Efron et al. calls it apparent error rate or resubstitution rate. It's not very good since we use our data (xi,yi) to fit f. This results in err¯¯¯¯¯¯¯ being downward biased. You want to know how well your model f does in predicting new values.

Often we use cross-validation as a simple way to estimate the expected extra-sample prediction error (how well does our model do on data not in our training set?).
Err=E[L(Y,f(X))]
A popular way to do this is to do K-fold cross-validation. Split your data into K groups (e.g. 10). For each group k, fit your model on the remaining K−1 groups and test it on the kth group. Our cross-validated extra-sample prediction error is just the average
ErrCV=1N∑i=1NL(yi,f−κ(i)(xi))
where κ is some index function that indicates the partition to which observation i is allocated and f−κ(i)(xi) is the predicted value of xi using data not in the κ(i)th set.




This estimator is approximately unbiased for the true prediction error when K=N and has larger variance and is more computationally expensive for larger K. So once again we see the bias–variance trade-off at play.

Instead of cross-validation we could use the bootstrap to estimate the extra-sample prediction error. Bootstrap resampling can be used to estimate the sampling distribution of any statistic. If our training data is X=(x1,…,xN), then we can think of taking B bootstrap samples (with replacement) from this set Z1,…,ZB where each Zi is a set of N samples. Now we can use our bootstrap samples to estimate extra-sample prediction error:
Errboot=1B∑b=1B1N∑i=1NL(yi,fb(xi))


where fb(xi) is the predicted value at xi from the model fit to the bth bootstrap dataset. Unfortunately, this is not a particularly good estimator because bootstrap samples used to produce fb(xi) may have contained xi. The leave-one-out bootstrap estimator offers an improvement by mimicking cross-validation and is defined as:
Errboot(1)=1N∑i=1N1|C−i|∑b∈C−iL(yi,fb(xi))
where C−i is the set of indices for the bootstrap samples that do not contain observation i, and |C−i| is the number of such samples. Errboot(1) solves the overfitting problem, but is still biased (this one is upward biased). The bias is due to non-distinct observations in the bootstrap samples that result from sampling with replacement. The average number of distinct observations in each sample is about 0.632N (see this answer for an explanation of why Why on average does each bootstrap sample contain roughly two thirds of observations?). To solve the bias problem, Efron and Tibshirani proposed the 0.632 estimator:
Err.632=0.368err¯¯¯¯¯¯¯+0.632Errboot(1)
where
err¯¯¯¯¯¯¯=1N∑i=1NL(yi,f(xi))
is the naïve estimate of prediction error often called training error. The idea is to average a downward biased estimate and an upward biased estimate.


However, if we have a highly overfit prediction function (i.e. err¯¯¯¯¯¯¯=0) then even the .632 estimator will be downward biased. The .632+ estimator is designed to be a less-biased compromise between err¯¯¯¯¯¯¯ and Errboot(1).
Err.632+=(1−w)err¯¯¯¯¯¯¯+wErrboot(1)
with
w=0.6321−0.368RandR=Errboot(1)−err¯¯¯¯¯¯¯γ−err¯¯¯¯¯¯¯
where γ is the no-information error rate, estimated by evaluating the prediction model on all possible combinations of targets yi and predictors xi.

γ=1N2∑i=1N∑j=1NL(yi,f(xj))
.

Here R measures the relative overfitting rate. If there is no overfitting (R=0, when the Errboot(1)=err¯¯¯¯¯¯¯) this is equal to the .632 estimator.

那我個人實驗目前比較採用的方式是:

設定最少要 bagging 出 (k / 2) + 1 的 feature, 才比較有顯著結果, K 為原本的 feature 數量

或者另外一個常見設定是 square(k)


因為重複採樣的關係, 平均來講, 每棵大約會有 1/3 training data 採樣不到

所以收集這些 data, 最後等到 Forest 建立完成之後, 將這些 data 餵進去判斷, 最後得出錯誤率

這方式稱為 Out-Of-Bag (OOB)

其實 Random Forest 是一個很 heuristic 的演算法

他還有很多需要被決定的參數, 像是, 我到底要用幾棵樹

well, 笨一點的方式就是從 1棵, 2棵...到 n 棵, 棵一路建立上去

然後計算他相對的 OOB, 要是發現 OOB 沒在下降, 那就差不多了